In [1]:
import numpy as np
import pandas as pd
import csv
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import classification_report

In [2]:
from sklearn.metrics import confusion_matrix

In [3]:
import seaborn as sns
import matplotlib.pyplot as plt

In [4]:
file_path = r"data/ciciot2023_dataset.csv"
df = pd.read_csv(file_path, header=0, index_col=0).drop([0]).reset_index(drop=True)

In [5]:
df['label'].mask(df['label'] == 'BenignTraffic', int(0), inplace=True)
df['label'].mask(df['label'] == 'DDoS-TCP_Flood', int(1), inplace=True)
df['label'].mask(df['label'] == 'DDoS-UDP_Flood', int(1), inplace=True)

In [6]:
for col in df.columns:
    df[col] = pd.to_numeric(df[col])

Split and Scale Dataset

In [8]:
train, valid, test = np.split(df.sample(frac=1),[int(0.6*len(df)), int(0.8*len(df))])

In [9]:
def scale(dataframe):
    X = dataframe[dataframe.columns[:-1]].values
    y = dataframe[dataframe.columns[-1]].values

    scaler = StandardScaler()
    X = scaler.fit_transform(X)

    data = np.hstack((X, np.reshape(y, (-1,1)))) 

    return data, X, y

In [10]:
train, x_train, y_train = scale(train)
valid, x_valid, y_valid = scale(test)
test, x_test, y_test = scale(test)

In [11]:
svm_model = SVC(kernel='linear')
svm_model = svm_model.fit(x_train, y_train)

In [12]:
y_pred = svm_model.predict(x_test)

In [ ]:
print(classification_report(y_test, y_pred, digits=4))

In [ ]:
coefficients = svm_model.coef_[0]
intercept = svm_model.intercept_[0]
hyperplane_equation = ' + '.join([f'({coefficients[i]:.3f} * x{i+1})' for i in range(len(coefficients))]) + f' + ({intercept:.3f}) = 0'
print("Hyperplane equation:")
print(hyperplane_equation)

In [15]:
cm = confusion_matrix(y_test, y_pred)
cm_normalized = cm / cm.sum(axis=1, keepdims=True)

In [ ]:
sns.heatmap(cm_normalized, annot=True, fmt="f", cmap="Blues")
plt.xlabel("Predicted Labels")
plt.ylabel("True Labels")
plt.title("Confusion Matrix")
plt.show()